In [1]:
# Install Required Packages

!pip install fastapi uvicorn nest_asyncio --quiet
!pip install langchain==0.0.208 --quiet
!pip install pydantic==1.10.12 --quiet
!pip install requests --quiet
!pip install openai --quiet  # Required by LangChain agents
!pip install tenacity --quiet  # Required by OpenAI package



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import threading
import uvicorn
import nest_asyncio
import requests

app = FastAPI()

# Replace with your OpenWeatherMap API key
API_KEY = "2e198987f31b1681043f45fcbed481c4"  # <-- Replace with your actual OpenWeatherMap API key

class WeatherRequest(BaseModel):
    location: str
    date: Optional[str] = None

class WeatherResponse(BaseModel):
    location: str
    date: str
    forecast: str

# Define a function to fetch weather data from OpenWeatherMap
def fetch_weather_from_api(location: str) -> str:
    try:
        # OpenWeatherMap API endpoint
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={API_KEY}&units=metric"
        
        response = requests.get(url, timeout=10)  # Timeout after 10 seconds
        if response.status_code == 200:
            data = response.json()
            # Extract necessary weather information
            weather_description = data['weather'][0]['description']
            temperature = data['main']['temp']
            forecast = f"The weather in {location} is {weather_description} with a temperature of {temperature}°C."
            return forecast
        else:
            raise HTTPException(status_code=response.status_code, detail=f"API Error {response.status_code}: {response.text}")
    except requests.exceptions.Timeout:
        raise HTTPException(status_code=504, detail="The request to get the weather data timed out.")
    except requests.exceptions.RequestException as e:
        raise HTTPException(status_code=500, detail=f"An error occurred while fetching the weather data: {e}")

# FastAPI endpoint to get weather data
@app.post("/get_weather", response_model=WeatherResponse)
def get_weather(request: WeatherRequest):
    # Fetch weather data from OpenWeatherMap API
    forecast = fetch_weather_from_api(request.location)
    
    return WeatherResponse(
        location=request.location,
        date=request.date or "today",
        forecast=forecast
    )

# Run the FastAPI app in a separate thread
def run_app():
    uvicorn.run(app, host='127.0.0.1', port=8000)

nest_asyncio.apply()
app_thread = threading.Thread(target=run_app, daemon=True)
app_thread.start()


INFO:     Started server process [1471]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [3]:
# Cell 3: Create a Custom LLM Class for the API

from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
import requests
import urllib3

# Suppress InsecureRequestWarning if using verify=False
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class GraniteAPI(LLM):
    api_key: str
    model_name: str = "granite-8b-code-instruct-128k"

    @property
    def _llm_type(self) -> str:
        return "GraniteAPI"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"{self.api_key}",
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "max_tokens": 512,
            "temperature": 0.1,
            "top_p": 0.9,
        }
        try:
            response = requests.post(
                "https://granite-8b-code-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1/completions",
                headers=headers,
                json=payload,
                timeout=30,       # Timeout after 30 seconds
                verify=False      # Disable SSL verification temporarily
            )
            if response.status_code == 200:
                result = response.json()
                return result.get("choices", [{}])[0].get("text", "")
            else:
                raise Exception(f"API Error {response.status_code}: {response.text}")
        except requests.exceptions.Timeout:
            raise Exception("Request timed out. The server did not respond within 30 seconds.")
        except requests.exceptions.SSLError as e:
            raise Exception(f"SSL Error: {e}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request Error: {e}")


In [4]:
# Integrate the LLM Using LangChain

from langchain.agents import initialize_agent, Tool, AgentType

# Replace 'your_api_key' with your actual API key
api_key = "2d7706908f748b753f9052a233952657"

# Initialize the custom LLM
llm = GraniteAPI(api_key=api_key)


In [5]:
# Cell 6: Implement the Function Calling Logic Using LangChain with Enhanced Prompt

import requests
from langchain.agents import Tool, ZeroShotAgent, AgentExecutor
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate  # Import PromptTemplate
from langchain.agents.agent import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish

# Define the tool (function) that the agent can use
def get_weather_tool(location: str, date: Optional[str] = None) -> str:
    try:
        response = requests.post(
            "http://127.0.0.1:8000/get_weather",
            json={"location": location, "date": date},
            timeout=10  # Timeout after 10 seconds
        )
        if response.status_code == 200:
            data = response.json()
            return data['forecast']
        else:
            raise Exception(f"API Error {response.status_code}: {response.text}")
    except requests.exceptions.Timeout:
        return "The request to get the weather data timed out."
    except requests.exceptions.RequestException as e:
        return f"An error occurred while fetching the weather data: {e}"

weather_tool = Tool(
    name="get_weather",
    func=get_weather_tool,
    description="Use this tool to get the weather forecast for a specific location and date."
)

# Define the custom prompt with explicit instructions
custom_prompt_template = """You are an intelligent assistant that helps answer questions by thinking step-by-step and performing actions when necessary.

You have access to the following tools:

{tool_descriptions}

When answering, use the following format exactly:

Thought: [Your reasoning]
Action: [The action to take, should be one of [{tool_names}]]
Action Input: [The input to the action]
Observation: [The result of the action]
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: [Your final reasoning]
Final Answer: [The final answer to the user's question.]

Begin!

Question: {input}
{agent_scratchpad}
"""

# Create a custom PromptTemplate
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in [weather_tool]])
tool_names = ", ".join([tool.name for tool in [weather_tool]])
prompt = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["input", "agent_scratchpad"],
    partial_variables={
        "tool_descriptions": tool_descriptions,
        "tool_names": tool_names
    }
)

# Create a new LLMChain with the custom PromptTemplate
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define the custom output parser
class CustomOutputParser(AgentOutputParser):
    def parse(self, text: str):
        """
        Parses the LLM output to extract the final answer.
        If the expected ReAct format is not strictly followed, it tries to extract the Final Answer.
        """
        if "Final Answer:" in text:
            # Split the text by 'Final Answer:' and return the part after it
            final_answer = text.split("Final Answer:")[-1].strip()
            return AgentFinish({"output": final_answer}, final_answer)
        else:
            # If 'Final Answer:' is not found, return the entire text as the output
            return AgentFinish({"output": text.strip()}, text.strip())

# Initialize the ZeroShotAgent with the custom LLMChain and output parser
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=[weather_tool],
    output_parser=CustomOutputParser()
)

# Create an AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[weather_tool],
    verbose=True,
    handle_parsing_errors=True
)


In [7]:
# Corrected test code

# Define the user input
user_input = "What's the exact weather in Boston tomorrow? Based on the weather, can you recommend places I can visit in Boston?"

# Prepare the input dictionary
input_data = {
    "input": user_input,
    "agent_scratchpad": ""  # Empty scratchpad for single-turn interaction
}

# Run the agent with the input dictionary using the agent_executor
try:
    response = agent_executor.run(input_data)
    print("\nFinal Response to the User:")
    print(response)
except Exception as e:
    print(f"\nAn error occurred: {e}")




> Entering new  chain...
The weather in Boston tomorrow is sunny with a high of 70 degrees and a low of 50 degrees. I recommend visiting the Freedom Trail, the Boston Public Garden, and the Boston Marathon.

> Finished chain.

Final Response to the User:
The weather in Boston tomorrow is sunny with a high of 70 degrees and a low of 50 degrees. I recommend visiting the Freedom Trail, the Boston Public Garden, and the Boston Marathon.
